## palabras
modelo.get_topic(topic_id)

## Documentos
modelo.get_document_info(docs)


In [1]:
import pandas as pd
import pickle
from IPython.display import display, HTML
import numpy as np
import ast

## Seleccionar los mejores resultados 

In [2]:
with open('resultado/resultados_modelos.pkl', 'rb') as archivo:
    resultados = pickle.load(archivo)
    
# Para bigramas    
# with open('resultados_modelos_bi.pkl', 'rb') as archivo:
#     resultados = pickle.load(archivo)

In [3]:
resultados["bertopic"]['resultados_ejecuciones']

[{'adjusted_rand_score': 0.04978071700704035,
  'normalized_mutual_info_score': 0.2685511262380395,
  'confusion_matrix': array([[  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         ...,
         [ 89,   1,   0, ...,   0,   0,   0],
         [ 23,   3,  23, ...,   0,   0,   0],
         [155,  22,  17, ...,   0,   0,   0]], dtype=int64),
  'fowlkes_mallows_score': 0.18978723404628486,
  'bcubed_precision': 0.23930699728444366,
  'bcubed_recall': 0.31544237404301906,
  'bcubed_f1': 0.1796902769786475},
 {'adjusted_rand_score': 0.05509724969263894,
  'normalized_mutual_info_score': 0.277259851088923,
  'confusion_matrix': array([[  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         ...,
         [ 79,   0,  47, ...,   0,   0,   0],
         [ 37,  23,   1, ...,   0,   0,   0],
         [157,  41,   4, ...,   0,   0,   0]]

In [4]:
lista_resultados = resultados["bertopic"]['resultados_ejecuciones']

# Crear un DataFrame de pandas
df = pd.DataFrame(lista_resultados)

# Eliminar la columna 'confusion_matrix' ya que no se puede representar directamente en HTML
df = df.drop(columns=['confusion_matrix'])

# Convertir el DataFrame a HTML
tabla_html = df.to_html(index=False)

# Imprimir la tabla HTML
display(HTML(tabla_html))

adjusted_rand_score,normalized_mutual_info_score,fowlkes_mallows_score,bcubed_precision,bcubed_recall,bcubed_f1
0.049781,0.268551,0.189787,0.239307,0.315442,0.179690
0.055097,0.277260,0.193812,0.246696,0.333570,0.187214
0.039985,0.260739,0.181293,0.235398,0.308867,0.169332
0.077310,0.302101,0.214924,0.272291,0.350189,0.212269
0.040542,0.268091,0.188438,0.248229,0.319417,0.171908
0.040542,0.268091,0.188438,0.248229,0.319417,0.171908
0.040542,0.268091,0.188438,0.248229,0.319417,0.171908
0.040542,0.268091,0.188438,0.248229,0.319417,0.171908
0.040542,0.268091,0.188438,0.248229,0.319417,0.171908
0.040542,0.268091,0.188438,0.248229,0.319417,0.171908


In [5]:
# Calcular la media para cada fila
medias = df[['adjusted_rand_score', 'normalized_mutual_info_score', 'bcubed_f1']].mean(axis=1)

# Añadir las medias como una nueva columna al DataFrame
df['Media'] = medias

df_ordenado = df.sort_values(by='Media',ascending=False)

# Crear un nuevo dataframe solo con las medias
df_medias = df_ordenado[['Media']]

# Convertir el DataFrame a HTML
tabla_html_medias = df_medias.to_html()

display(HTML(tabla_html_medias))

,Media
3,0.197227
1,0.173190
0,0.166007
4,0.160180
5,0.160180
6,0.160180
7,0.160180
8,0.160180
9,0.160180
2,0.156685


In [3]:
etiquetas_predichas=resultados["bertopic"]['etiquetas_predichas'][3]
# Para bigramas
# etiquetas_predichas=resultados["bertopic"]['etiquetas_predichas'][1]

## Obtener las palabras más relevantes de cada tópico 

In [8]:
#Por tamaño del modelo más de un 1 GB no se incluye como recurso contactar con el autor si se necesita
with open('modelos_topicos/bertopic_preprocesado_ejecucion_3.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)
# with open('modelos_topicos_bi/bertopic_ejecucion_1.pkl', 'rb') as archivo:
#     modelo = pickle.load(archivo)

In [9]:
topic_info = modelo.get_topic_info()

In [4]:
topic_info =topic_info[topic_info["Topic"] != -1]
top_20_topics = topic_info.sort_values(by="Count", ascending=False).head(20)
for topic_id in top_20_topics["Topic"]:
    topic_words = modelo.get_topic(topic_id)[:15] # Obtiene las 15 primeras palabras
    print(f"Tópico {topic_id}: {topic_words}")
    print("***"*20)

Tópico 0: [('luz', 0.06968958398437251), ('farola', 0.06965495401756874), ('fundido', 0.05870990053784502), ('matricular', 0.05414097794672406), ('semaforo', 0.05136468255486845), ('punto', 0.04716184412555638), ('apagado', 0.04067950497730758), ('reparacion', 0.02824950795748706), ('iluminacion', 0.024727124375343433), ('vertical', 0.02390060760297067)]
************************************************************
Tópico 1: [('bache', 0.034664841847174756), ('avenida', 0.016826883391385907), ('rama', 0.01617028646146506), ('reparacion', 0.015886951903490893), ('calle', 0.01483455188446729), ('solicitar', 0.014032703336561512), ('tapar', 0.013913880782853784), ('paseo', 0.011775894862521069), ('semaforo', 0.010619830844358687), ('arbol', 0.009493951984649177)]
************************************************************
Tópico 2: [('perro', 0.05629408296450111), ('limpieza', 0.021961965525059832), ('parque', 0.016410554076367746), ('excremento', 0.01628490310994549), ('calle', 0.0156505

In [13]:
import pandas as pd

def generar_excel_topicos_palabras_bertopic(modelo, topic_info, output_excel_path="topicos_palabras_bertopic.xlsx"):

    # Filtrar los tópicos válidos (eliminamos el tópico residual 0)
    topic_info = topic_info[topic_info["Topic"] != -1]

    # Excluir el primer tópico (tópico residual) y obtener los 20 siguientes
    top_20_topics = topic_info.sort_values(by="Count", ascending=False)

    # Lista para almacenar los datos que se agregarán al Excel
    data = []

    # Recorrer los 20 tópicos más relevantes (excepto el tópico residual)
    for topic_id in top_20_topics["Topic"]:
        # Obtener las 15 primeras palabras del tópico
        topic_words = modelo.get_topic(topic_id)[:15]  # Obtiene las primeras 15 palabras
        # Extraer solo las palabras, no los pesos
        topic_words = [word[0] for word in topic_words]
        # Crear un diccionario con el ID del tópico y las palabras asociadas
        data.append({"Tópico ID": topic_id, "Palabras": ", ".join(topic_words)})

    # Crear un DataFrame de pandas a partir de los datos
    df = pd.DataFrame(data)

    # Exportar el DataFrame a un archivo Excel
    df.to_excel(output_excel_path, index=False)

    print(f"El archivo Excel con los tópicos y palabras se ha exportado a '{output_excel_path}'.")


In [14]:
generar_excel_topicos_palabras_bertopic(modelo, topic_info, output_excel_path="topicos_palabras_bertopic.xlsx")

El archivo Excel con los tópicos y palabras se ha exportado a 'topicos_palabras_bertopic.xlsx'.


## Obtener las documentos más relevantes de cada tópico

In [15]:
import pandas as pd

def generar_excel_documentos_relevantes_bertopic(modelo, documentos, output_excel_path="documentos_relevantes_bertopic.xlsx"):
 
    # Obtener la asignación de tópicos para cada documento
    document_info = modelo.get_document_info(documentos)
    
    # Filtrar el tópico residual (-1)
    document_info = document_info[document_info['Topic'] != -1]

    # Contar cuántos documentos hay por tópico
    topic_counts = document_info['Topic'].value_counts()

    # Obtener los 20 tópicos con más documentos
    top_20_topics = topic_counts.head(20).index

    # Lista para almacenar los datos a exportar
    data = []

    # Recorrer los 20 tópicos con más documentos
    for topic_id in top_20_topics:
        # Obtener los documentos relacionados con el tópico y su probabilidad
        topic_documents = document_info[document_info['Topic'] == topic_id]
        
        # Ordenar los documentos por la probabilidad de pertenecer al tópico (de mayor a menor)
        topic_documents_sorted = topic_documents.sort_values(by="Probability", ascending=False)
        
        # Tomar los 5 documentos más relevantes
        top_5_documents = topic_documents_sorted.head(5)

        # Añadir los documentos más relevantes al DataFrame
        for index, row in top_5_documents.iterrows():
            documento = documentos[row.name]  # Obtener el documento original (sin procesar)
            probabilidad = row["Probability"]  # Probabilidad de que el documento pertenezca al tópico
            data.append({"Tópico ID": topic_id, "Documento": documento[:300], "Probabilidad": probabilidad})  # Solo los primeros 300 caracteres

    # Crear un DataFrame a partir de los datos recopilados
    df = pd.DataFrame(data)

    # Exportar el DataFrame a un archivo Excel
    df.to_excel(output_excel_path, index=False)

    print(f"Los 5 documentos más relevantes para los 20 tópicos con más documentos se han exportado a '{output_excel_path}'.")


In [18]:
docs=df_preprocesado['texto']

In [19]:
generar_excel_documentos_relevantes_bertopic(modelo, docs)

Los 5 documentos más relevantes para los 20 tópicos con más documentos se han exportado a 'documentos_relevantes_bertopic.xlsx'.
